In [20]:
import os
import pandas as pd
import requests
from statistics import mean
import time

In [23]:
def find_places_and_distances(api_key, latitude, longitude, radius=1000, place_type='university'):
    results = []
    page_token = None

    while True:
        nearby_search_endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        location = f"{latitude},{longitude}"

        nearby_search_params = {
            'location': location,
            'radius': radius,
            'type': place_type,
            'key': api_key,
            'pagetoken': page_token
        }

        nearby_response = requests.get(nearby_search_endpoint, params=nearby_search_params)
        response_json = nearby_response.json()

        if response_json.get('status') != 'OK':
            print(f"Error or no results for {place_type} at location ({latitude}, {longitude}): {response_json.get('status')}")
            break

        results.extend(response_json.get('results', []))

        page_token = response_json.get('next_page_token')
        if not page_token:
            break

        time.sleep(2)  # Necessary to wait for the next page token to become valid

    return results

def calculate_distances(api_key, latitude, longitude, places):
    if not places:
        return None, None

    location = f"{latitude},{longitude}"
    max_elements_per_request = 25  # Maximum destinations per request to avoid exceeding API limits
    distances = []

    for i in range(0, len(places), max_elements_per_request):
        chunk = places[i:i + max_elements_per_request]
        destinations = [f"{place['geometry']['location']['lat']},{place['geometry']['location']['lng']}" for place in chunk]

        distance_matrix_params = {
            'origins': location,
            'destinations': '|'.join(destinations),
            'key': api_key,
            'mode': 'walking'
        }

        distance_matrix_endpoint = "https://maps.googleapis.com/maps/api/distancematrix/json"
        distance_matrix_response = requests.get(distance_matrix_endpoint, params=distance_matrix_params)
        distance_matrix_json = distance_matrix_response.json()

        if distance_matrix_json.get('status') != 'OK':
            print(f"Distance Matrix API error for location ({latitude}, {longitude}): {distance_matrix_json.get('status')}")
            return None, None

        distances.extend([element['distance']['value'] for element in distance_matrix_json['rows'][0]['elements'] if 'distance' in element])

    avg_distance = mean(distances) if distances else None
    shortest_distance = min(distances) if distances else None

    return avg_distance, shortest_distance

# Read CSV file
df = pd.read_csv('boba_shops.csv')  # Replace with your CSV file path

# Define API key
api_key = os.environ["googleplaces_api"]

# Process each bubble tea shop
results = []
for index, row in df.iterrows():
    shop_name = row['Place Name']
    shop_latitude = row['Latitude']
    shop_longitude = row['Longitude']
    shop_place_id = row['Place ID']

    universities = find_places_and_distances(api_key, shop_latitude, shop_longitude, place_type='university')
    highschools = find_places_and_distances(api_key, shop_latitude, shop_longitude, place_type='school')

    avg_distance_uni, shortest_distance_uni = calculate_distances(api_key, shop_latitude, shop_longitude, universities) if universities else (None, None)
    avg_distance_hs, shortest_distance_hs = calculate_distances(api_key, shop_latitude, shop_longitude, highschools) if highschools else (None, None)

    results.append({
        'Bubble Tea Shop': shop_name,
        'Latitude': shop_latitude,
        'Longitude': shop_longitude,
        'Place ID': shop_place_id,
        'Number of Nearby Colleges': len(universities),
        'Nearby Colleges': [uni['name'] for uni in universities] if universities else [],
        'Average Distance to Colleges (m)': avg_distance_uni,
        'Shortest Distance to Colleges (m)': shortest_distance_uni,
        'Number of Nearby Schools': len(highschools),
        'Nearby Schools': [hs['name'] for hs in highschools] if highschools else [],
        'Average Distance to Schools (m)': avg_distance_hs,
        'Shortest Distance to Schools (m)': shortest_distance_hs,
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

results_df


Error or no results for university at location (45.492157, -73.709819): ZERO_RESULTS
Error or no results for university at location (45.5812866, -73.6405367): ZERO_RESULTS
Error or no results for university at location (45.5281968, -73.6481237): ZERO_RESULTS
Error or no results for university at location (45.44045240000001, -73.6222118): ZERO_RESULTS


,Bubble Tea Shop,Latitude,Longitude,Place ID,Number of Nearby Colleges,Nearby Colleges,Average Distance to Colleges (m),Shortest Distance to Colleges (m),Number of Nearby Schools,Nearby Schools,Average Distance to Schools (m),Shortest Distance to Schools (m)
0,Bubble Tea Shop,45.536106,-73.613523,ChIJrRPobmMZyUwRmzUtZysr570,7,[University of Montreal School of Public Healt...,870.000000,417.0,60,"[École La Petite-Patrie, Pavillon Saint-Jean-D...",723.466667,120
1,Bubble Tea Shop,45.498627,-73.570964,ChIJk0Ay_ZwbyUwRtjPsppB80xM,60,"[McGill University, École de technologie supér...",794.750000,319.0,60,"[EC Montreal English Language School, MLS, F.A...",666.650000,6
2,Bubble Tea Shop,45.501355,-73.570814,ChIJh1KLHbAbyUwR5DRuYkp1Gvg,60,"[McGill University, O'Sullivan College, Desaut...",729.716667,0.0,60,"[University of Quebec at Montreal, F.A.C.E. El...",601.350000,4
3,OLa Boba Tea,45.583184,-73.542436,ChIJe-mGnqcdyUwRDR7LWixnrUY,4,"[voy k5, College Institute Trudeau Internation...",475.000000,249.0,24,"[Scout, Espace Voix | Singing Lessons And Musi...",791.291667,59
4,Chatime,45.508589,-73.572116,ChIJ1xjTVUgayUwRvxmlGV6LkLU,60,"[McGill University, Cegep of Old Montreal, Uni...",753.333333,109.0,60,"[University of Quebec at Montreal, Montreal Hi...",633.583333,162
5,The Alley Montreal,45.494823,-73.577654,ChIJ0XpAvrgbyUwRLq2_-QxhoO4,60,"[Collège of Montréal, Collège LaSalle, O'Sulli...",600.233333,12.0,60,"[Collège of Montréal, EC Montreal English Lang...",479.783333,3
6,Bubble Tea Shop,45.495379,-73.579280,ChIJuQHWPh0byUwRlgTAaNZ68yE,60,"[Collège of Montréal, Collège LaSalle, O'Sulli...",572.333333,59.0,60,"[Collège of Montréal, Académie Michèle-Provost...",467.066667,59
7,Gong Cha,45.525748,-73.581550,ChIJOXWPWI0byUwRI-q2TjxNmGQ,4,[GERIQ - Groupe d'études et de recherche sur l...,393.000000,0.0,60,"[Laurier Elementary School, Yoga Sangha, Nurse...",702.183333,37
8,Bubble Tea Shop,45.452951,-73.645885,ChIJb3yKdrAXyUwRxKieAJfUEKU,6,"[Concordia University - Loyola Campus, Centre ...",961.166667,782.0,35,"[Loyola High School, Collège Prep Internationa...",871.028571,94
9,Real Fruit Bubble Tea,45.502673,-73.571714,ChIJLWhYybIbyUwROq0s6LLmi2w,60,"[McGill University, O'Sullivan College, Univer...",762.600000,307.0,60,"[University of Quebec at Montreal, F.A.C.E. El...",647.400000,199


In [25]:
# Save the results back to a new CSV
results_df.to_csv('schools.csv', index=False)